In [ ]:
# ─── 1. INSTALLATION ───────────────────────────────────────────────────────
!sudo apt-get update -y
!sudo apt-get install -y tesseract-ocr poppler-utils
!pip install pytesseract tqdm pymupdf


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,927 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,546 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages 

In [ ]:
# ─── 2. IMPORTS ────────────────────────────────────────────────────────────
import os, zipfile, re, json
import cv2
import numpy as np
import pandas as pd
import pytesseract
from tqdm import tqdm
from google.colab import files
import fitz  # PyMuPDF

In [ ]:
# ─── 3. TÉLÉVERSEMENT & EXTRACTION ────────────────────────────────────────
uploaded = files.upload()
zip_path = next(iter(uploaded))
extract_dir = "extracted_images"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z: z.extractall(extract_dir)

# Si tu as des PDF, convertir chaque page en PNG
for root, _, files_ in os.walk(extract_dir):
  for f in files_:
    if f.lower().endswith(".pdf"):
      doc = fitz.open(os.path.join(root, f))
      for i in range(doc.page_count):
        pix = doc[i].get_pixmap(dpi=300)
        out = os.path.join(root, f"{os.path.splitext(f)[0]}_p{i}.png")
        pix.save(out)
      doc.close()

Saving batch_1 (2).zip to batch_1 (2).zip


In [ ]:
# ─── 4. COLLECTE DES IMAGES ────────────────────────────────────────────────
supported_images = []
for root, _, files_ in os.walk(extract_dir):
  for f in files_:
    if f.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff")):
      supported_images.append(os.path.join(root, f))
print(f"🔍 {len(supported_images)} images trouvées.")

🔍 1489 images trouvées.


In [ ]:
# ─── 5. PRÉTRAITEMENT OPTIMISÉ PAR LOTS ─────────────────────────────────
preprocessed_dir = "preprocessed"
os.makedirs(preprocessed_dir, exist_ok=True)

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise IOError(f"Impossible de lire {img_path}")
    gray    = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    binary  = cv2.adaptiveThreshold(
                  blurred, 255,
                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                  cv2.THRESH_BINARY, 11, 2)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, np.ones((1,1), np.uint8))
    out = os.path.join(preprocessed_dir, os.path.basename(img_path))
    cv2.imwrite(out, cleaned)
    return out

batch_size = 100
preprocessed_paths = []
for i in range(0, len(supported_images), batch_size):
    batch = supported_images[i:i+batch_size]
    for p in tqdm(batch, desc=f"Batch {i//batch_size+1}", unit="img"):
        try:
            preprocessed_paths.append(preprocess_image(p))
        except Exception as e:
            print("⚠️", e)
print(f"✅ {len(preprocessed_paths)} images prétraitées.")

Batch 15: 100%|██████████| 89/89 [00:05<00:00, 16.87img/s]

✅ 1489 images prétraitées.


In [ ]:
# ─── 6. OCR AVEC TESSERACT ────────────────────────────────────────────────
custom_oem_psm = r'--oem 3 --psm 6'
texts = []
for p in tqdm(preprocessed_paths, desc="OCR", unit="img"):
    img = cv2.imread(p)
    texts.append(pytesseract.image_to_string(img, config=custom_oem_psm, lang="fra+eng"))

OCR: 100%|██████████| 1489/1489 [1:08:56<00:00,  2.78s/img]


In [ ]:
# ─── 7. EXTRACTION STRUCTURÉE VIA REGEX ─────────────────────────────────
def extract_fields(text, fichier):
    texte = text.replace("\n", " ").strip()  # Nettoyer le texte en remplaçant les sauts de ligne par des espaces

    # ===== Extraction par regex =====
    numero_facture = re.search(r"(?:invoice\s*(?:no)?)[:\-]?\s*([0-9]{5,})", texte, re.IGNORECASE)
    date = re.search(r"(?:date\s*of\s*issue)?[:\-]?\s*(\b(?:\d{2}[\/\-]\d{2}[\/\-]\d{4}|\d{4}[\/\-]\d{2}[\/\-]\d{2})\b)", texte, re.IGNORECASE)
    client = re.search(r"(?:Client\s*:?)\s*([\w\s\-]+(?:\n[\w\s\-]+)?)", texte, re.IGNORECASE)
    sous_total = re.search(r"(?:Net\s+worth|Sous[-\s]?total)[^\d]{0,10}(\d{1,3}(?:[.,]\d{2})?)", texte, re.IGNORECASE)
    tva = re.search(r"(?:VAT|TVA)[^\d]{0,10}(\d{1,3}(?:[.,]\d{2})?)", texte, re.IGNORECASE)
    montant_total = re.search(r"(?:Gross\s+worth|Total)[^\d]{0,10}(\d{1,3}(?:[.,]\d{2})?)", texte, re.IGNORECASE)

    # Nettoyage client (si saut de ligne)
    client_name = client.group(1).replace("\n", " ").strip() if client else "Non trouvé"

    # Résultats extraits et formatés
    result = {
        "Fichier": fichier,
        "Numéro de facture": numero_facture.group(1) if numero_facture else "Non trouvé",
        "Date": date.group(1) if date else "Non trouvée",
        "Client": client_name,
        "Sous-total": sous_total.group(1) if sous_total else "Non trouvé",
        "TVA": tva.group(1) if tva else "Non trouvée",
        "Montant total": montant_total.group(1) if montant_total else "Non trouvé"
    }

    return result

# ─── 8. EXTRACTION DES DONNÉES DES FACTURES ───────────────────────────────
results = []
for idx, texte in enumerate(texts):
    fichier = f"facture_{idx + 1}.pdf"  # Exemple de nom de fichier
    result = extract_fields(texte, fichier)
    results.append(result)

# ─── 9. EXPORT CSV / JSON & APERÇU ────────────────────────────────────────
df = pd.DataFrame(results)
df.to_csv("resultats_factures.csv", index=False)
with open("resultats_factures.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("✅ Fichiers enregistrés :")
print("- resultats_factures.csv")
print("- resultats_factures.json")

# Afficher les 5 premières lignes pour vérifier
display(df.head(5))
# Sauvegarde CSV
df = pd.DataFrame(results)
df.to_csv("factures_extract.csv", index=False)
files.download("factures_extract.csv")

# Sauvegarde JSON
with open("factures_extract.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)
files.download("factures_extract.json")

✅ Fichiers enregistrés :
- resultats_factures.csv
- resultats_factures.json


Fichier Numéro de facture        Date  \
0  facture_1.pdf          10963794  08/09/2012   
1  facture_2.pdf          10596025  12/31/2020   
2  facture_3.pdf          35977350  04/01/2016   
3  facture_4.pdf          77596491  01/31/2021   
4  facture_5.pdf          46939079  04/26/2018   

                                              Client  Sous-total          TVA  \
0     Oconnor Group Pena-Schaefer 7440 Bowen Key Apt  Non trouvé  Non trouvée   
1                                              Hobbs  Non trouvé  Non trouvée   
2  Richardson PLC Alvarado and Sons 0026 Beck Cir...  Non trouvé  Non trouvée   
3                                               Tran  Non trouvé  Non trouvée   
4                                   Thomas Inc Clark  Non trouvé  Non trouvée   

  Montant total  
0             1  
1             1  
2             1  
3             1  
4            10

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>